## Load Data

In [0]:
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data as data
import torchvision as tv
import torchvision.models as models
from PIL import Image
import glob
import os
import time
from torch.optim import lr_scheduler
import copy


class DatasetManager:
    
    def __init__(self, dataset = 'cifar10', percent_data = 10.0, percent_val = 20.0, data_path = './data'):
        
        # 'dataset' can be 'hymenoptera', 'cifar10', or 'cifar100'.
        # 'percent_data' is the percentage of the full training set to be used.
        # 'percent_val' is the percentage of the *loaded* training set to be used as validation data.
        
        self.dataset = dataset
        self.data_path = data_path
        self.percent_data = percent_data
        self.percent_val = percent_val
        
        if self.dataset == 'hymenoptera':

            self.transform = tv.transforms.Compose([
                tv.transforms.RandomResizedCrop(224),
                tv.transforms.RandomHorizontalFlip(),
                tv.transforms.ToTensor(),
                tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
            
        elif self.dataset == 'cifar10' or self.dataset == 'cifar100':

            self.transform = tv.transforms.Compose([
                tv.transforms.RandomResizedCrop(224),
                tv.transforms.RandomHorizontalFlip(),
                tv.transforms.ToTensor(),
                tv.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
        
        return
    
    
    def ImportDataset(self, batch_size=5):
        
        self.batch_size = batch_size
        
        if self.dataset == 'hymenoptera':
        
            self.trainset = tv.datasets.ImageFolder(root=self.data_path,
                             transform=self.transform)
        
        # todo
        
        elif self.dataset == 'cifar10':

            self.trainset = tv.datasets.CIFAR10(root=self.data_path, train=True,
                                        download=True, transform=self.transform)

            self.testset = tv.datasets.CIFAR10(root=self.data_path, train=False,
                                       download=True, transform=self.transform)
        
        elif self.dataset == 'cifar100':

            self.trainset = tv.datasets.CIFAR100(root=self.data_path, train=True,
                                        download=True, transform=self.transform)

            self.testset = tv.datasets.CIFAR100(root=self.data_path, train=False,
                                       download=True, transform=self.transform)
             
        self.SplitData();
        self.GenerateLoaders();
                
        return
    
    
    def SplitData(self):
        
        len_full = self.trainset.__len__()
        len_train = int(np.round(len_full*self.percent_data/100.0))
        
        _, self.trainset = torch.utils.data.random_split(self.trainset, (len_full-len_train, len_train))
        
        len_val = int(np.round(len_train*self.percent_val/100.0))
        len_train = len_train - len_val
        
        self.valset, self.trainset = torch.utils.data.random_split(self.trainset, (len_val, len_train))
         
        len_full_test = self.testset.__len__()
        len_test = int(np.round(len_full_test*self.percent_data/100.0))
        
        _, self.testset = torch.utils.data.random_split(self.testset, (len_full_test-len_test, len_test))

        print('\nFull training set size: {}'.format(len_full))
        print('Full test set size: {}'.format(len_full_test))
        print('\nActive training set size: {}'.format(len_train))
        print('Active validation set size: {}'.format(len_val))
        print('Active test set size: {}'.format(len_test))
        
        return
    
    
    def GenerateLoaders(self):
        
        self.train_loader = torch.utils.data.DataLoader(self.trainset, batch_size=self.batch_size,
                                          shuffle=True, num_workers=0)
        self.val_loader = torch.utils.data.DataLoader(self.valset, batch_size=self.batch_size,
                                          shuffle=True, num_workers=0)
        self.test_loader = torch.utils.data.DataLoader(self.testset, batch_size=self.batch_size,
                                          shuffle=True, num_workers=0)          
            
        return


In [3]:
# Import data

dat = DatasetManager('cifar10', 10.0, 20.0)
dat.ImportDataset(5)



0it [00:00, ?it/s]

100%|█████████▉| 170205184/170498071 [00:43<00:00, 3373051.55it/s]

Files already downloaded and verified

Full training set size: 50000
Full test set size: 10000

Active training set size: 4000
Active validation set size: 1000
Active test set size: 1000


## Pruning Functions

In [0]:
# Class that contains various settings pertaining to how filters are pruned
class UnitPruningSettings:
    
    def __init__(self, idx_layer, idx_filter, N_prune = 1, p = 2):
        
        self.N_prune = N_prune # Number of filters allowed to be pruned in one pass
        self.idx_filter = idx_filter # Indices of the N_prune filters
        self.idx_layer = idx_layer # Current layer under consideration
        self.p = p # p-norm to use when computing which filters to remove
        
        return


In [0]:
# These functions were adapted from https://github.com/jacobgil/pytorch-pruning/blob/master/prune.py

def replace_layers(model, i, idx, layers):
	if i in idx:
		return layers[idx.index(i)]
	return model[i]

# Function to prune a given convolution layer in the model provided.
# Input "idx_layers" is the global index of the convolution layer to be pruned.
# Input "prune_settings" is a data structure containing information on how pruning is performed.
def PruneConvLayers(model, prune_settings):
    
    # Strategy: in order to prune a particular layer, the output of the previous layer 
    # and the inputs to the next layer must also be altered accordingly.
	
    # Extract pruning settings for convenience
    # Note that "N_prune" *consecutive* filters will get pruned
#     N_prune = prune_settings.N_prune
    N_prune = 1
    idx_filter = prune_settings.idx_filter
    idx_layer = prune_settings.idx_layer
    
    # Extract the layer of the model currently being pruned
    _, conv = list(model.features._modules.items())[idx_layer]
#     _, conv = model.features._modules.items()(idx_layer)
    
    # To keep track of the succeeding convolution layer
    next_conv = None
    offset = 1
    
    # Figure out how many layers after this one are NOT conv layers, in order to skip pruning them
    while idx_layer + offset < len(model.features._modules.items()):
        
        res =  list(model.features._modules.items())[idx_layer + offset]
        if isinstance(res[1], torch.nn.modules.conv.Conv2d):
            next_name, next_conv = res
            break
        offset = offset + 1
    
    # Create a new, replacement conv layer to remove a given number of filters.
    # The rest of its settings should remain the same as the original conv layer.
    new_conv = torch.nn.Conv2d(in_channels = conv.in_channels,
                               out_channels = conv.out_channels - N_prune,
			                   kernel_size = conv.kernel_size,
                               stride = conv.stride,
                               padding = conv.padding,
                               dilation = conv.dilation,
                               groups = conv.groups,
                               bias = True)
    
    new_conv.bias = conv.bias
    
    # Copy over the weights to the new conv layer, except the ones corresponding to the filter to be removed
    old_weights = conv.weight.data.cpu().numpy()
    new_weights = new_conv.weight.data.cpu().numpy()

    # This copies the set of filters up to and excluding the filters to be removed
    new_weights[: idx_filter, :, :, :] = old_weights[: idx_filter, :, :, :]

    # This copies the filters after and excluding the filters to be removed
    new_weights[idx_filter :, :, :, :] = old_weights[idx_filter + N_prune :, :, :, :]

    # Update weight data of the new conv layer
#     new_conv.weight.data = torch.from_numpy(new_weights).cuda()
    new_conv.weight.data = torch.from_numpy(new_weights)

    # Now do the same thing for biases
    old_biases = conv.bias.data.cpu().numpy()

    new_biases = np.zeros(shape = (old_biases.shape[0] - N_prune), dtype = np.float32)
    new_biases[:idx_filter] = old_biases[:idx_filter]
    new_biases[idx_filter :] = old_biases[idx_filter + N_prune :]
#     new_conv.bias.data = torch.from_numpy(new_biases).cuda()
    new_conv.bias.data = torch.from_numpy(new_biases)
    
    # If there is a succeeding conv layer, adjust its input units and weights accordingly
    if next_conv != None:
        
        next_new_conv = torch.nn.Conv2d(in_channels = next_conv.in_channels - N_prune,
                                        out_channels =  next_conv.out_channels,
                                        kernel_size = next_conv.kernel_size,
                                        stride = next_conv.stride,
                                        padding = next_conv.padding,
                                        dilation = next_conv.dilation,
                                        groups = next_conv.groups,
                                        bias = True)
        
        next_new_conv.bias = next_conv.bias

        old_weights = next_conv.weight.data.cpu().numpy()
        new_weights = next_new_conv.weight.data.cpu().numpy()

        new_weights[:, : idx_filter, :, :] = old_weights[:, : idx_filter, :, :]
        new_weights[:, idx_filter : , :, :] = old_weights[:, idx_filter + N_prune :, :, :]
#         next_new_conv.weight.data = torch.from_numpy(new_weights).cuda()
        next_new_conv.weight.data = torch.from_numpy(new_weights)

        next_new_conv.bias.data = next_conv.bias.data

        # Update the actual model by replacing the existing filters with the new ones
        features = torch.nn.Sequential(
                *(replace_layers(model.features, i, [idx_layer, idx_layer + offset], \
                    [new_conv, next_new_conv]) for i, _ in enumerate(model.features)))
        del model.features
        del conv

        model.features = features
    
    else:

        # This is the last conv layer. This affects the first linear layer of the classifier.
        model.features = torch.nn.Sequential(
                *(replace_layers(model.features, i, [idx_layer], \
                    [new_conv]) for i, _ in enumerate(model.features)))
        idx_layer = 0
        old_linear_layer = None

        for _, module in model.classifier._modules.items():
            if isinstance(module, torch.nn.Linear):
                old_linear_layer = module
                break
            idx_layer = idx_layer + 1

        if old_linear_layer == None:
            raise BaseException("No linear layer found in classifier.")
        params_per_input_channel = old_linear_layer.in_features / conv.out_channels

        new_linear_layer = \
            torch.nn.Linear(int(old_linear_layer.in_features - params_per_input_channel), 
                old_linear_layer.out_features)

        old_weights = old_linear_layer.weight.data.cpu().numpy()
        new_weights = new_linear_layer.weight.data.cpu().numpy()	 	

        new_weights[:, : int(idx_filter * params_per_input_channel)] = \
            old_weights[:, : int(idx_filter * params_per_input_channel)]
        new_weights[:, int(idx_filter * params_per_input_channel) :] = \
            old_weights[:, int((idx_filter + N_prune) * params_per_input_channel) :]

        new_linear_layer.bias.data = old_linear_layer.bias.data

#         new_linear_layer.weight.data = torch.from_numpy(new_weights).cuda()
        new_linear_layer.weight.data = torch.from_numpy(new_weights)

        classifier = torch.nn.Sequential(
            *(replace_layers(model.classifier, i, [idx_layer], \
                [new_linear_layer]) for i, _ in enumerate(model.classifier)))

        del model.classifier
        del next_conv
        del conv
        model.classifier = classifier
        
    return model
        

In [6]:
# Test pruning

model = models.vgg16(pretrained=True)
model.train()

# Pruning setup
prune_settings = UnitPruningSettings(28, 10, 1)

t0 = time.time()
model = PruneConvLayers(model, prune_settings)
print ("Pruning took {} s".format(time.time() - t0))


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth

0it [00:00, ?it/s]
2547712it [00:00, 25456629.93it/s]
12214272it [00:00, 32677392.33it/s]
19587072it [00:00, 39226650.06it/s]
26484736it [00:00, 45053929.43it/s]
31784960it [00:00, 45784712.39it/s]
38797312it [00:00, 51101246.77it/s]
48824320it [00:00, 59910976.07it/s]
57548800it [00:00, 66119314.51it/s]
66174976it [00:00, 70873167.42it/s]
74604544it [00:01, 74337518.02it/s]
84066304it [00:01, 72106890.28it/s]
93757440it [00:01, 78030950.78it/s]
104054784it [00:01, 84142701.83it/s]
112902144it [00:01, 85160872.30it/s]
121962496it [00:01, 86715579.01it/s]
130859008it [00:01, 86276254.90it/s]
139649024it [00:01, 75768614.52it/s]
148078592it [00:01, 78134040.57it/s]
157065216it [00:02, 81314383.15it/s]
165412864it [00:02, 75706618.39it/s]
173211648it [00:02, 72029681.04it/s]
182870016it [00:02, 77967071.14it/s]
192643072it [00:02, 82991088.71it/s]
202588160it [00:02, 87320604.0

Pruning took 1.234318494796753 s


## Training Function

In [0]:
def train_model(model, dat, criterion, optimizer, scheduler, num_epochs=25):
    
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                scheduler.step()
                dataloader = dat.train_loader
                dataset_size = dat.trainset.__len__()
                
                model.train()  # Set model to training mode
                
            else:
                
                model.eval()   # Set model to evaluate mode
                dataloader = dat.val_loader
                dataset_size = dat.valset.__len__()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader:
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model


## Baseline Model Setup

In [0]:
model = models.vgg16(pretrained=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


## Train Baseline Model

In [22]:
# Import data
dat = DatasetManager('cifar10', 1.0, 20.0)
dat.ImportDataset(5)

model.train()

model = train_model(model, dat, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

Files already downloaded and verified
Files already downloaded and verified

Full training set size: 50000
Full test set size: 10000

Active training set size: 400
Active validation set size: 100
Active test set size: 100
Epoch 0/24
----------
train Loss: 2.1566 Acc: 0.2325
val Loss: 2.0811 Acc: 0.2800

Epoch 1/24
----------
train Loss: 2.1084 Acc: 0.2500
val Loss: 1.9544 Acc: 0.3100

Epoch 2/24
----------
train Loss: 2.0179 Acc: 0.2650
val Loss: 2.0884 Acc: 0.2400

Epoch 3/24
----------
train Loss: 1.9010 Acc: 0.3075
val Loss: 2.0639 Acc: 0.2800

Epoch 4/24
----------
train Loss: 1.8655 Acc: 0.3150
val Loss: 1.8821 Acc: 0.2800

Epoch 5/24
----------
train Loss: 1.7540 Acc: 0.3675
val Loss: 1.9376 Acc: 0.3300

Epoch 6/24
----------
train Loss: 1.5314 Acc: 0.4350
val Loss: 1.9539 Acc: 0.2900

Epoch 7/24
----------
train Loss: 1.4907 Acc: 0.4600
val Loss: 1.6439 Acc: 0.3300

Epoch 8/24
----------
train Loss: 1.3321 Acc: 0.5350
val Loss: 1.7334 Acc: 0.3200

Epoch 9/24
----------
train Los

## Pruned Model Setup - Single Filter

In [9]:
model = models.vgg16(pretrained=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Pruning setup
prune_settings = UnitPruningSettings(28, 10, 1)

t0 = time.time()
model = PruneConvLayers(model, prune_settings)
print ("Pruning took {} s".format(time.time() - t0))

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

Pruning took 1.4709546566009521 s


## Train Pruned Model - Single Filter

In [25]:
# Import data
dat = DatasetManager('cifar10', 1.0, 20.0)
dat.ImportDataset(5)

model.train()

model = train_model(model, dat, criterion, optimizer, exp_lr_scheduler, num_epochs=25)


Files already downloaded and verified
Files already downloaded and verified

Full training set size: 50000
Full test set size: 10000

Active training set size: 400
Active validation set size: 100
Active test set size: 100
Epoch 0/24
----------
train Loss: 2.8876 Acc: 0.0875
val Loss: 2.3552 Acc: 0.1900

Epoch 1/24
----------
train Loss: 2.4806 Acc: 0.1275
val Loss: 2.2240 Acc: 0.1800

Epoch 2/24
----------
train Loss: 2.3176 Acc: 0.1900
val Loss: 2.1939 Acc: 0.2000

Epoch 3/24
----------
train Loss: 2.2381 Acc: 0.1875
val Loss: 2.1577 Acc: 0.2000

Epoch 4/24
----------
train Loss: 2.1509 Acc: 0.2350
val Loss: 2.1055 Acc: 0.1900

Epoch 5/24
----------
train Loss: 2.1236 Acc: 0.2425
val Loss: 2.0256 Acc: 0.2600

Epoch 6/24
----------
train Loss: 1.8307 Acc: 0.3250
val Loss: 1.8670 Acc: 0.2900

Epoch 7/24
----------
train Loss: 1.7686 Acc: 0.3500
val Loss: 1.9131 Acc: 0.3200

Epoch 8/24
----------
train Loss: 1.7535 Acc: 0.3400
val Loss: 1.8303 Acc: 0.3400

Epoch 9/24
----------
train Los

## Functions to Compute Filter Metrics

In [11]:
# Function to compute the p-norm of weights in all filters of a given layer.
# The list of norms are returned in a list in the same order as that in which filters of that layer are stored.
def ComputeConv2DFilterNorms(model, idx_layer, p):
    
    # Extract the layer of the model currently being considered
    _, conv = list(model.features._modules.items())[idx_layer]
    
    weights = conv.weight.data

    # Compute norms of each filter
    norms = weights.norm(p, dim=2)
    norms = norms.norm(p, dim=2)
    norms = norms.norm(p, dim=1)
    
    return norms

# Test
norms_1 = ComputeConv2DFilterNorms(model, 0, 1)
norms_2 = ComputeConv2DFilterNorms(model, 0, 2)
norms_inf = ComputeConv2DFilterNorms(model, 0, float('inf'))

print(norms_1.size())
# print(norms_1)

# Compute values and indices of the k smallest norms
k = 2
n1_botk, ind_n1_botk = torch.topk(norms_1, k, 0, largest=False, sorted=True, out=None)
print(n1_botk)
print(len(ind_n1_botk))
print(ind_n1_botk)


torch.Size([64])
tensor([1.0097, 1.9274])
2
tensor([54, 50])


In [0]:
# Function to iterate through all conv2D layers of the network and determine 
# filters to be pruned, and then carry out the pruning.
def PruneAllConv2DLayers(model, prune_settings):
    
    # Extract pruning settings for convenience
    # Note that "N_prune" *consecutive* filters will get pruned
    N_prune = prune_settings.N_prune
    p = prune_settings.p
    
    # Find the N_prune filters to remove
    ii = 0
    while ii < len(model.features._modules.items()):
        
        res = list(model.features._modules.items())[ii]
        
        if isinstance(res[1], torch.nn.modules.conv.Conv2d):
            
            # Compute values and indices of the N_prune smallest norms
            norms = ComputeConv2DFilterNorms(model, ii, p)
            n_botk, ind_botk = torch.topk(norms, N_prune, 0, largest=False, sorted=True, out=None)
            
            # Prune filters one at a time (todo: prune all at once)
            prune_settings.idx_layer = ii
            
            jj = 0
            while jj < len(ind_botk):
                prune_settings.idx_filter = ind_botk[jj]
                print("Pruning filter {} in layer {}".format(ind_botk[jj], ii))
                model = PruneConvLayers(model, prune_settings)
                jj = jj + 1
                
        ii = ii + 1
    
    return model



## Pruned Model Setup - Several Filters

In [13]:
# Test pruning all layers

model = models.vgg16(pretrained=False)
model.train()

# Pruning setup
prune_settings = UnitPruningSettings(28, 10, N_prune = 2, p = 2)

t0 = time.time()
model = PruneAllConv2DLayers(model, prune_settings)
print ("Pruning took {} s".format(time.time() - t0))


Pruning filter 0 in layer 0
Pruning filter 60 in layer 0
Pruning filter 5 in layer 2
Pruning filter 16 in layer 2
Pruning filter 47 in layer 5
Pruning filter 28 in layer 5
Pruning filter 42 in layer 7
Pruning filter 54 in layer 7
Pruning filter 4 in layer 10
Pruning filter 29 in layer 10
Pruning filter 10 in layer 12
Pruning filter 238 in layer 12
Pruning filter 169 in layer 14
Pruning filter 218 in layer 14
Pruning filter 371 in layer 17
Pruning filter 283 in layer 17
Pruning filter 374 in layer 19
Pruning filter 91 in layer 19
Pruning filter 86 in layer 21
Pruning filter 377 in layer 21
Pruning filter 26 in layer 24
Pruning filter 480 in layer 24
Pruning filter 23 in layer 26
Pruning filter 358 in layer 26
Pruning filter 351 in layer 28
Pruning filter 131 in layer 28
Pruning took 3.5376181602478027 s


## Train Pruned Model - Several Filters

In [0]:
# Import data
dat = DatasetManager('cifar10', 1.0, 20.0)
dat.ImportDataset(5)

model.train()

model = train_model(model, dat, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

Files already downloaded and verified
Files already downloaded and verified

Full training set size: 50000
Full test set size: 10000

Active training set size: 400
Active validation set size: 100
Active test set size: 100
Epoch 0/24
----------
